In [1]:
import pandas as pd
import numpy as np

In [13]:
df = pd.DataFrame(list(zip([1,2,3,4], ['2010-04-11', '2010-04-23', '2010-05-03', '2010-05-05'])), columns=['col1', 'date'])
df.date = pd.to_datetime(df.date)
df = df.set_index('date')
df

,col1
date,
2010-04-11,1
2010-04-23,2
2010-05-03,3
2010-05-05,4


In [15]:
df.resample('M').sum()

,col1
date,
2010-04-30,3
2010-05-31,7


In [18]:
df = df.reset_index()
df.groupby(df['date'].dt.to_period('M')).agg(my_sum=('col1', 'sum'))

,my_sum
date,
2010-04,3
2010-05,7


In [2]:
df = pd.read_csv('click_stream.csv', header=None, names = ['ID','page','date', 'device', 'gender'], parse_dates= ['date'])
df.head()

,ID,page,date,device,gender
0,313593,1_home_page,2015-02-26,Desktop,Female
1,468315,1_home_page,2015-02-21,Desktop,Male
2,264005,1_home_page,2015-03-25,Desktop,Female
3,290784,1_home_page,2015-03-14,Desktop,Male
4,639104,1_home_page,2015-01-03,Desktop,Female


In [3]:
df_pivot = df.pivot_table(index=df.date.dt.to_period('M'), columns=['gender', 'device', 'page'], values='ID', aggfunc='count')
df_pivot

gender       Female                                                           \
device      Desktop                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01        7546          3851            625                          33   
2015-02        7448          3800            605                          26   
2015-03        7512          3707            162                           6   
2015-04        7491          3733            138                           9   

gender                                                                        \
device       Mobile                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01        3868          3062            618                          76   
2015-02        3751          3020            630                          60   
2015-03        3738           747            175                          20   
2015-04        3721           756            147                          11   

gender         Male                                                           \
device      Desktop                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01        7504          3678            596                          27   
2015-02        7602          3832            586                          28   
2015-03        7538          3663            134                           8   
2015-04        7559          3836            164                          13   

gender                                                                        
device       Mobile                                                           
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page  
date                                                                          
2015-01        3682          2963            551                          53  
2015-02        3799          3035            591                          59  
2015-03        3812           762            160                          10  
2015-04        3829           755            148                          13

In [6]:
df_pivot.columns.levels[0]

Index(['Female', 'Male'], dtype='object', name='gender')

In [7]:
df_pivot.columns.levels[1]

Index(['Desktop', 'Mobile'], dtype='object', name='device')

In [4]:
df_pivot[('Female', 'Desktop')]

page,1_home_page,2_search_page,3_payment_page,4_payment_confirmation_page
date,,,,
2015-01,7546,3851,625,33
2015-02,7448,3800,605,26
2015-03,7512,3707,162,6
2015-04,7491,3733,138,9


In [8]:
for gender in df_pivot.columns.levels[0]:
    for device in df_pivot.columns.levels[1]:
        df_pivot[(gender, device)] = df_pivot[(gender, device)].apply(lambda x: round(x / x[0] * 100, 2), axis=1).copy()
df_pivot  

gender       Female                                                           \
device      Desktop                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01       100.0         51.03           8.28                        0.44   
2015-02       100.0         51.02           8.12                        0.35   
2015-03       100.0         49.35           2.16                        0.08   
2015-04       100.0         49.83           1.84                        0.12   

gender                                                                        \
device       Mobile                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01       100.0         79.16          15.98                        1.96   
2015-02       100.0         80.51          16.80                        1.60   
2015-03       100.0         19.98           4.68                        0.54   
2015-04       100.0         20.32           3.95                        0.30   

gender         Male                                                           \
device      Desktop                                                            
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01       100.0         49.01           7.94                        0.36   
2015-02       100.0         50.41           7.71                        0.37   
2015-03       100.0         48.59           1.78                        0.11   
2015-04       100.0         50.75           2.17                        0.17   

gender                                                                        
device       Mobile                                                           
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page  
date                                                                          
2015-01       100.0         80.47          14.96                        1.44  
2015-02       100.0         79.89          15.56                        1.55  
2015-03       100.0         19.99           4.20                        0.26  
2015-04       100.0         19.72           3.87                        0.34

In [9]:
pd.concat([df_pivot[('Female', 'Desktop')], df_pivot[('Male', 'Desktop')]], axis=1, keys=['Female', 'Male'])

Female                                                           \
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page   
date                                                                           
2015-01       100.0         51.03           8.28                        0.44   
2015-02       100.0         51.02           8.12                        0.35   
2015-03       100.0         49.35           2.16                        0.08   
2015-04       100.0         49.83           1.84                        0.12   

               Male                                                           
page    1_home_page 2_search_page 3_payment_page 4_payment_confirmation_page  
date                                                                          
2015-01       100.0         49.01           7.94                        0.36  
2015-02       100.0         50.41           7.71                        0.37  
2015-03       100.0         48.59           1.78                        0.11  
2015-04       100.0         50.75           2.17                        0.17